In [1]:
%matplotlib inline
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# --- ATLAS style ---
ROOT.gROOT.LoadMacro("/home/kalelc/research/atlasstyle/AtlasStyle.C")
ROOT.gROOT.LoadMacro("/home/kalelc/research/atlasstyle/AtlasUtils.C")
ROOT.gROOT.LoadMacro("/home/kalelc/research/atlasstyle/AtlasLabels.C")
ROOT.SetAtlasStyle()

ROOT.EnableImplicitMT()

# --- Load Delphes ---
delphes_dir = "/home/kalelc/research/Delphes-3.5.0/"
status = ROOT.gSystem.Load(delphes_dir + "libDelphes.so")
if status != 0:
    raise RuntimeError("Error: Could not load libDelphes.so")
print("libDelphes.so loaded successfully!")

# Input ROOT file
fdisplaced = delphes_dir + "displacedPhoton/displaced_half_dark_events.root"
fprompt = delphes_dir + "displacedPhoton/prompt_half_dark_events.root"
df_displaced = ROOT.RDataFrame("Delphes", fdisplaced)
df_prompt = ROOT.RDataFrame("Delphes", fprompt)


Applying ATLAS style settings...

libDelphes.so loaded successfully!


In [2]:
# --- Convert photons to pandas ---
displaced_particle_dict = df_displaced.AsNumpy([
    "Event.Number",
    "Photon.PT",
    "Photon.Eta",
    "Photon.Phi",
    "Photon.T",
    "Photon_size"
])

# Flatten event numbers (scalars, not 1-element arrays)
event_numbers = [arr[0] for arr in displaced_particle_dict["Event.Number"]]
num_photons = displaced_particle_dict["Photon_size"]

# Build DataFrame
displaced_particle_df = pd.DataFrame({
    "EventNumber": event_numbers,
    "NumPhotons": num_photons,
    "PT": displaced_particle_dict["Photon.PT"],
    "Eta": displaced_particle_dict["Photon.Eta"],
    "Phi": displaced_particle_dict["Photon.Phi"],
    "T": displaced_particle_dict["Photon.T"],

})

# Explode so each particle gets its own row
displaced_particle_df = displaced_particle_df.explode([ "PT", "Eta", "Phi", "T"], ignore_index=True)

displaced_particle_df = displaced_particle_df.sort_values(by="EventNumber").reset_index(drop=True)

# Filter to only events with at least one photon
displaced_particle_df = displaced_particle_df[displaced_particle_df["NumPhotons"] > 0]

displaced_particle_df

,EventNumber,NumPhotons,PT,Eta,Phi,T
1,1,1,56.019508,0.323366,-0.583914,0.0
2,2,1,45.905617,-0.255704,-2.874639,0.0
3,3,2,33.105965,1.750007,-0.319087,0.0
4,3,2,59.00642,0.209569,-1.36837,0.0
5,4,1,45.928295,-0.23588,-2.851681,0.0
...,...,...,...,...,...,...
7516,5986,2,80.102142,-0.432837,3.01262,0.0
7519,5989,1,201.658356,-0.656675,2.958543,0.0
7521,5991,1,200.032211,-0.652832,2.956488,0.0
7525,5995,1,48.730186,2.376185,0.558778,0.0


In [3]:
# --- Convert photons to pandas ---
prompt_particle_dict = df_prompt.AsNumpy([
    "Event.Number",
    "Photon.PT",
    "Photon.Eta",
    "Photon.Phi",
    "Photon.T",
    "Photon_size"
])

# Flatten event numbers (scalars, not 1-element arrays)
event_numbers = [arr[0] for arr in prompt_particle_dict["Event.Number"]]
num_photons = prompt_particle_dict["Photon_size"]

# Build DataFrame
prompt_particle_df = pd.DataFrame({
    "EventNumber": event_numbers,
    "NumPhotons": num_photons,
    "PT": prompt_particle_dict["Photon.PT"],
    "Eta": prompt_particle_dict["Photon.Eta"],
    "Phi": prompt_particle_dict["Photon.Phi"],
    "T": prompt_particle_dict["Photon.T"],

})

# Explode so each particle gets its own row
prompt_particle_df = prompt_particle_df.explode([ "PT", "Eta", "Phi", "T"], ignore_index=True)

prompt_particle_df = prompt_particle_df.sort_values(by="EventNumber").reset_index(drop=True)

# Filter to only events with at least one photon
prompt_particle_df = prompt_particle_df[prompt_particle_df["NumPhotons"] > 0]

prompt_particle_df

,EventNumber,NumPhotons,PT,Eta,Phi,T
0,0,1,38.24575,1.586421,-0.336175,0.0
1,1,1,57.159962,0.326268,-0.548867,0.0
2,2,1,46.879234,-0.249339,-2.934141,0.0
3,3,2,39.459129,1.583968,-0.348154,0.0
4,3,2,57.284534,0.321744,-0.541536,0.0
...,...,...,...,...,...,...
9031,5993,1,202.12999,-0.630256,2.954215,0.0
9032,5994,1,13.271816,-0.198787,-3.09077,0.0
9033,5995,1,52.756062,2.306507,0.537381,0.0
9034,5996,1,51.825382,2.309587,0.529647,0.0


In [4]:
# Find the intersection of EventNumber values in both dataframes
shared_event_numbers = set(displaced_particle_df["EventNumber"]).intersection(prompt_particle_df["EventNumber"])

# Filter both dataframes to include only shared events
displaced_particle_df = displaced_particle_df[displaced_particle_df["EventNumber"].isin(shared_event_numbers)].reset_index(drop=True)
prompt_particle_df = prompt_particle_df[prompt_particle_df["EventNumber"].isin(shared_event_numbers)].reset_index(drop=True)

displaced_particle_df, prompt_particle_df

(      EventNumber  NumPhotons          PT       Eta       Phi    T
 0               1           1   56.019508  0.323366 -0.583914  0.0
 1               2           1   45.905617 -0.255704 -2.874639  0.0
 2               3           2   33.105965  1.750007 -0.319087  0.0
 3               3           2    59.00642  0.209569  -1.36837  0.0
 4               4           1   45.928295  -0.23588 -2.851681  0.0
 ...           ...         ...         ...       ...       ...  ...
 5062         5986           2   84.618362 -0.654476 -0.177844  0.0
 5063         5986           2   80.102142 -0.432837   3.01262  0.0
 5064         5989           1  201.658356 -0.656675  2.958543  0.0
 5065         5991           1  200.032211 -0.652832  2.956488  0.0
 5066         5995           1   48.730186  2.376185  0.558778  0.0
 
 [5067 rows x 6 columns],
       EventNumber  NumPhotons          PT       Eta       Phi    T
 0               1           1   57.159962  0.326268 -0.548867  0.0
 1               2  